# DProvDB Experimental Evaluation on EQW (BFS) Task

We would like to understand DProvDB's performance via empirical study.

## End-to-end Comparison

First we would like to perform an end-to-end comparison of DProvDB against baseline systems.

We have the following baselines:

- DProvDB minus additiveGM (Vanilla mechanism in our paper)
- Chorus
- DProvDB minus cached views (i.e., enabling Chorus with Provenance table)
- Simulating PrivateSQL 

Note that the key idea in PrivateSQL is to generate synopses for pre-determined views and answer queries using synopses.
The overall privacy budget is split to generate the synopses in advance.
One prominent way to allocate budget to views, as mentioned in PrivateSQL paper, is fair allocation.
That is to split the budget w.r.t the sensitivity of the views s.t. the expected error on each view is the same.
Since PrivateSQL system is not open source, we use our system with a specific setting (DProvDB minus additiveGM, with predetermined view constraints) to simulate PrivateSQL.

Each experiment is run for 4 times, we plot the mean and the variance as evaluation results.

In [ ]:
# load package and fonts

import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

sns.set(font="Helvetica", rc={"figure.figsize":(4, 3)})
sns.set_theme(style="white")
sns.set_style('ticks')
sns.set_context("paper", font_scale=1.3, rc={"lines.linewidth": 1.75})

1) RRQ, over Adult dataset

In [ ]:
# load data

report = pd.read_csv('../data/EQW_adult_end_to_end.csv', sep=';')

report = report.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
report.columns = report.columns.str.strip()


# processing list strings

def parse_list(s):
    if isinstance(s, str) and s.startswith('List'):
        return [float(x) for x in s[5: -1].split(",")]
    else:
        return s

report = report.applymap(parse_list)

report.loc[(report["viewConstraintFlag"] == "static fixed: 1.0") & (report["mechanism"] == "baseline"), "mechanism"] = "PrivateSQL"
report.loc[(report["viewConstraintFlag"] == "static fixed: 1.0") & (report["mechanism"] == "PrivateSQL"), "viewConstraintFlag"] = "dynamic fixed: 1.0"

report.head(5)

In [ ]:
# load data for tpc-h

report_tpc = pd.read_csv('../data/EQW_tpch_end_to_end.csv', sep=';')

report_tpc = report_tpc.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
report_tpc.columns = report_tpc.columns.str.strip()


report_tpc = report_tpc.applymap(parse_list)

report_tpc.loc[(report_tpc["viewConstraintFlag"] == "static fixed: 1.0") & (report_tpc["mechanism"] == "baseline"), "mechanism"] = "PrivateSQL"
report_tpc.loc[(report_tpc["viewConstraintFlag"] == "static fixed: 1.0") & (report_tpc["mechanism"] == "PrivateSQL"), "viewConstraintFlag"] = "dynamic fixed: 1.0"

report_tpc.head(5)

In [ ]:
# processing data

mechanism_mapping = {"aGM": "DProvDB", "baseline": "Vanilla", "Chorus": "Chorus", "ChorusP": "ChorusP"}

def processing_data_EQW(df, dataset, task="EQW_BFS", sample_freq = 12, mech_dt = ["aGM", "baseline", "Chorus", "ChorusP"]):
    
    filtered_df = df[(df["dataset"]==dataset) & (df["task"]==task)]
            
    ret_dt = {}

    for mech in mech_dt:
        sample_num = len(list(filtered_df[filtered_df['mechanism']==mech]['accountant'].values)[0])//sample_freq
    
        for i in range(sample_num):
            workload_index = i * int(filtered_df['totalNoOfQueries'].values[0]) / sample_num          
            ret_dt[(mechanism_mapping[mech], workload_index)] = [ l[i*sample_freq] for l in list(filtered_df[filtered_df['mechanism']==mech]['accountant'].values)]

    return ret_dt


# test
dt = processing_data_EQW(report_tpc, "tpch", "EQW_BFS")
# print(dt)


In [ ]:
# plotting

# get data
dt = processing_data_EQW(report, "adult", "EQW_BFS")

dt_tpc = processing_data_EQW(report_tpc, "tpch", "EQW_BFS")



sns.set(font="Helvetica", rc={"figure.figsize":(4, 3)})
sns.set_theme(style="white")
sns.set_style('ticks')
sns.set_context("paper", font_scale=1.4, rc={"lines.linewidth": 1.75})

# drawing figures
fig, axes = plt.subplots(ncols=2,figsize=(8, 3))

data = pd.Series(dt).reset_index()
data.columns = ['mechanisms', 'workload_index', 'budget']
data = data.explode("budget")
line = sns.lineplot(data=data, x='workload_index', y='budget', hue='mechanisms', ax=axes[0], linewidth=1.5, style="mechanisms", markers=True, markersize=9, dashes=True)
handles, labels = axes[0].get_legend_handles_labels()

sns.move_legend(axes[0], "lower center", bbox_to_anchor=(1.1, 1), ncol=4, title="Mechanisms", frameon=True,handles=handles[::-1], markerscale=2.3, fontsize=13, title_fontsize=15)
line.set(ylabel="Cumulative budget")
line.set(xlabel="workload index (Adult)")

# cur_indices = line.get_xticklabels()
# new_xticks = [int(i.get_text()) *12 for i in cur_indices[1:]]
# print(new_xticks)
# line.set_xticklabels(new_xticks)



data = pd.Series(dt_tpc).reset_index()
data.columns = ['mechanisms', 'workload_index', 'budget']
data = data.explode("budget")
line = sns.lineplot(data=data, x='workload_index', y='budget', hue='mechanisms', legend=False, ax=axes[1], linewidth=1.5, style="mechanisms", markers=True, markersize=9, dashes=True)
handles, labels = axes[0].get_legend_handles_labels()
axes[1].set(xlabel="workload index (TPC-H)")
axes[1].set(ylabel=None)


plt.savefig('end_to_end_EQW.pdf', dpi=600, bbox_inches='tight')

